# Create Azure Machine Learning Models

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../jobs/configuration.ipynb) 

- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a model from a local file
- Create a model from an mlflow model
- Create a model from cloud path
- Create a model from a run

**Motivations** - This notebook explains how to register a model in the Azure Machine Learning workspace.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
# %pip install azure-ml==0.0.139 --extra-index-url  https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Create a model
Azure ML models consist of the binary file(s) that represent a machine learning model and any corresponding metadata. Models can be created from a local file or directory. The created model will be tracked in the workspace under the specified name and version.

The `Model` class can be used to create a model. It accepts the following key parameters:
- `name` - Name of the model.
- `version` - Version of the model. If omitted, Azure ML will autogenerate a version.
- `path` - Local path to the model file(s). This can point to either a file or a directory.
- `type` - Storage format of the model. Applicable for no-code deployment scenarios. Allowed values are `custom_model`, `mlflow_model`, `triton_model`
- `description` - Description of the model.

## 2.1 Create a model from a local file
In this sample we will create a model from a local `pkl` file and specify the type of the model to be `custom_model`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="mlflow-model/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="local-file-example",
    description="Model created from local file.",
)
ml_client.models.create_or_update(file_model)

## 2.2 Create a model from an mlflow model
In this sample we will create a model from a local folder. We will also specify the type of the model to be `mlflow_model`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

mlflow_model = Model(
    path="mlflow-model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-example",
    description="MLflow model created from local path",
)
ml_client.create_or_update(mlflow_model)

## 2.3 Create a model from cloud path
In this sample we will create a model from a cloud path. We will also specify the format of the model to be `custom_model`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

cloud_model = Model(
    path=file_model.path,
    # The above line basically provides a path in the format "azureml://subscriptions/XXXXXXXXXXXXXXXX/resourceGroups/XXXXXXXXXXX/workspaces/XXXXXXXXXXX/datastores/workspaceblobstore/paths/model.pkl"
    # Users could also use,"azureml://datastores/workspaceblobstore/paths/model.pkl" as a shorthand to the same location
    name="cloud-path-example",
    type=AssetTypes.CUSTOM_MODEL,
    description="Model created from cloud path.",
)
ml_client.models.create_or_update(cloud_model)

## 2.4 Create a model from a run
In this sample we will create a model from a run. We will also specify the type of the model to be `mlflow_model`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path="azureml://subscriptions/XXXXXXXXXXXXXXXXXXXXX/resourceGroups/XXXXXXXXX/workspaces/XXXXXXXXXXXXX/jobs/XXXXXXXXXXXX/outputs/artifacts/paths/model/",
    # Users could also use, "azureml://jobs/XXXXXXXXXXXXXXXXXXXXXXXXX/outputs/artifacts/paths/model/" as a shorthand to the same location
    name="run-model-example",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
# ml_client.models.create_or_update(run_model)

# Next Steps
- Deploy a model to an Online Inferencing endpoint - [example](../../endpoints/online/managed/online-endpoints-simple-deployment.ipynb) 
- Deploy a model to a Batch Inferencing endpoint - [example](../../endpoints/batch/mnist-nonmlflow.ipynb)
